# 🧪 TP : Spécialisation ("Fine-tuning") d'un ResNet18
Dans ce TP, vous allez spécialiser un ResNet pré-entraîné sur ImageNet pour classifier des images de plantes.

Pour ce faire, vous allez utiliser :
- un GPU de [Google Colab](https://colab.research.google.com/) (vous aurez donc besoin d'un compte Google),
- la base d'images de plantes [PlantNet](https://meta-album.github.io/datasets/PLT_NET.html).

**Objectifs du TP :**
- Apprendre à spécialiser un ResNet18
- Entraîner 3 modèles : un ResNet spécialisé, un ResNet spécialisé mais où uniquement la dernière couche FC est optimisée, un ResNet entraîné "from scratch"
- Comparer les performances des 3 réseaux précédents

**REMARQUE : En cumulé, les 3 entraînements (en faisant 5 epochs pour chaque entraînement) durent environ 1h30.** 

## 📁 ÉTAPE 1 : Mise en place de PlantNet dans Google Colab

La base PlantNet a été prédécoupée en 3 morceaux train/validation/test. Les liens sont disponibles ci-après :
- [train](https://drive.google.com/file/d/1-N9C1YBeb71BfgX0JuRRyVGnIes2AsNC/view?usp=sharing)
- [test](https://drive.google.com/file/d/1jgfMjSFE6A_phBOiZZVtyDaliKDscG-G/view?usp=sharing)
- [val](https://drive.google.com/file/d/1qYyikdncPTQ_gLkyOTsA_k01o6taoDhT/view?usp=sharing)

**Il n'est pas nécessaire de télécharger les 3 fichiers zip, cliquez simplement sur "Ajouter un raccourci dans Drive". De cette manière, les 3 fichiers seront disponibles dans votre Drive et directement utilisables dans Google Colab.**

Dans Google Colab :
- ouvrez un nouveau notebook,
- dans le menu en haut à droite, aller sur "Change runtime type" et sélectionner T4 GPU.

In [ ]:
# Mount Google Drive to access dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip training and validation data into /tmp
!unzip -q "/content/drive/MyDrive/train.zip" -d "/tmp"
!unzip -q "/content/drive/MyDrive/val.zip" -d "/tmp"

À ce stade, la base PlantNet est prête à être utilisée.

## ⚙️ ÉTAPE 2 : Code utile pour la spécialisation d'un ResNet

Vous trouverez ci-après plusieurs morceaux de code facilitant la spécialisation d'un ResNet.

### Définition du `dataset` PyTorch
Un dataset PyTorch peut être créé en utilisant la classe `ImageFolder`. De plus, la normalisation des images en utilisant les statistiques d'ImageNet peut s'effectuer en utilisant la classe `transforms`. 

In [ ]:
import torchvision
import torchvision.transforms as transforms

# Image transforms for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Load dataset using ImageFolder
train_dataset = torchvision.datasets.ImageFolder(root=train_data_dir, transform=transform)
val_dataset = torchvision.datasets.ImageFolder(root=val_data_dir, transform=transform)

### Définition du `device`

In [ ]:
import torch
#define GPU as device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### Démarrer TensorBoard dans Google Colab

In [ ]:
#START TENSORBOARD

%load_ext tensorboard
%tensorboard --logdir=runs

### Fonction permettant de récupérer un ResNet18

In [ ]:
from torchvision.models import resnet18
def get_model(pretrained=True, freeze_backbone=False):
    model = resnet18(pretrained=pretrained)
    if freeze_backbone:
        for param in model.parameters():
            param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
    if freeze_backbone:
        for param in model.fc.parameters():
            param.requires_grad = True
    return model.to(device)

---
## ✅ Travail
- [ ] Afficher les images et les étiquettes de quelques éléments d'un minibatch
- [ ] Régler la taille d'un minibatch et du nombre de "workers" pour optimiser le temps de calcul
- [ ] Lancer 3 entraînements ("from scratch", "pre-trained frozen", "pretrained")
- [ ] Afficher l'évolution du taux de bonne classification pour chacun des trois apprentissages. Superposer les 3 courbes. Que constatez-vous ?
- [ ] Pour le meilleur modèle (sur l'ensemble de validation), créer un script permettant de charger les poids et tester ses performances sur la base de test.  